This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'TvZ5fzevTCiy5Pc_2MLq'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'
payload = {'start_date': '2020-02-24', 'end_date': '2020-02-25', 'api_key': API_KEY}

response = requests.get(url, params=payload)

responsejson = response.json()

print(responsejson)

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2020-02-24', 'end_date': '2020-02-25', 'frequency': 'daily', 'data': [['2020-02-25', 105.6, 106.4, 100.6, 101.2, None, 250020.0, 25613306.6, None, None, None], ['2020-02-24', 104.0, 105.5, 102.2, 104.7, None, 211543.0, 22050204.4, None, None, None]], 'collapse': None, 'order': None}}


In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

## 1. Collect data from the Franfurt Stock Exchange.

In [5]:
# Initialize url and call parameters
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'
payload = {'start_date': '2017-01-01', 'end_date': '2017-12-31', 'api_key': API_KEY}

response = requests.get(url, params=payload)

## 2. Convert the returned JSON object into a Python dictionary.

In [6]:
responsedata = response.json()

# Drill into the data from the API response
data = responsedata['dataset_data']['data']

# Define commonly used column names for the data list based on the column names field in the response data
column_names = responsedata['dataset_data']['column_names']
index_date = column_names.index('Date')
index_open = column_names.index('Open')
index_close = column_names.index('Close')
index_high = column_names.index('High')
index_low = column_names.index('Low')
index_volume = column_names.index('Traded Volume')

## 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [7]:
# Use a list comprehension to extract a list of opening prices. Filters for missing and non-numerical entries.
opens = [day[index_open] for day in data if type(day[index_open]) == float]

# Find and printthe min and max opening price
max_open = max(opens)
min_open = min(opens)
print('Maximum Opening: ${:.2f}'.format(max_open), ', Minimum Opening: ${:.2f}'.format(min_open))

Maximum Opening: $53.11 , Minimum Opening: $34.00


## 4. The largest change in any one day (based on High and Low price)

In [9]:
# Use a list comprehension to extract a list of daily price range
daily_range = [day[index_high]-day[index_low] for day in data if type(day[index_high]) == float and type(day[index_low] == float )]

# Find and print the maximum of the daily change
max_daily_change = max(daily_range)
print('Largest change in one day: ${:.2f}'.format(max_daily_change))

Largest change in one day: $2.81


## 5. The largest change between any two days (based on Closing Price)

In [10]:
# Use a list comprehension to create a list of closing prices
closing_prices = [day[index_close] for day in data if type(day[index_close]) == float]

# Use zip and a list comprehension to create a list of the change in closing price from the previous day
closing_change = [curr - prev for curr, prev in zip(closing_prices[1:], closing_prices[:-1])]

# Find and print the largest change
max_close_change = max(closing_change)
print('Largest change between two days: ${:.2f}'.format(max_close_change))

Largest change between two days: $2.56


## 6. The average daily trading volume during this year

In [11]:
# Use a list comprehension to extract a list of daily trading volume
trading_volume = [day[index_volume] for day in data if type(day[index_volume]) == float]

# Find and print the average trading volume
average_trading_volume = sum(trading_volume)/len(trading_volume)
print('Average daily trading volume: ${:.2f}'.format(average_trading_volume))

Average daily trading volume: $89124.34


## 7. The median trading volume during this year.

In [12]:
# Define function for taking the median of a list
def median( input_list ):
    """Returns the median of a list. Assumes list is numerical"""
    sorted_list = sorted(input_list)
    list_length = len(sorted_list)
    
    if (list_length%2 == 0) :
        # If the list has an even number of elements, return the average of the center two
        return (sorted_list[int(list_length/2)] + sorted_list[int(list_length/2-1)])/2
    else :
        # otherwise return the center element
        return (sorted_list[int((list_length-1)/2)])
    
# Find and print median of the trading volume list from previous question (6)
median_trading_volume = median(trading_volume)
print('Median daily trading volume: ${:.2f}'.format(median_trading_volume))

Median daily trading volume: $76286.00
